#### Example of using crosswalks between census tracts of different years

This notebook provides example code for using the output from the script in creating geojsons with multiple years of census data for the same tract. 

For simplicity's sake, in step 2 we transform the data into the frontend team's desired nested format first before using the cross walk on it. That way, if you transform your data into this format first, you can use the same functions from section 3 onward as-is.

In the future, if/when I fully automate this process, I will have a script where you can specify the census variables and it will do all of the work for you.

#### 0.) Set Census and Azure API credentials

```yaml

# e.g. in a file called api_info.yaml
azure: 
    connection-str:  # Sensitive info -- search Azure storage for the following storage account and container
    storage-account: 'pipelinemapping'
    container-name: 'std-geoms'
census:
    key: # Your personal Census API key
```

#### 1.) Download the crosswalks from the Azure container 

In [229]:
import json
import yaml
import pandas as pd 
import geopandas
import pygris 
from census import Census 
from utils import AzureBlobStorageManager
from logger import logger
import os
import sys
import numpy as np
import geopandas as gpd

## Configure APIs 
with open('api_info.yaml', 'r') as file: 
    data = yaml.full_load(file)
    census_api_key = data['census']['key']
    azure_conn_str = data['azure']['connection-str']
    azure_container_name = data['azure']['container-name']
    
c = Census(year=2020, key=census_api_key)
azure_manager = AzureBlobStorageManager(connection_str=azure_conn_str, 
                                        container_name=azure_container_name,
                                        download_dir='data')

In [8]:
## Download from Azure container (if required)
for fp in azure_manager.list_blobs(): 
    dl_path = os.path.join(azure_manager.download_dir, fp)
    if os.path.isfile(dl_path):
        logger.info(f'{fp} already exists in specified download directory ({azure_manager.download_dir})')
    else: 
        azure_manager.download_blob(fp)

2023-12-04 11:08:04,989 - logger - INFO - convert-ctracts_pct-area_2010-to-2020.json already exists in specified download directory (data)
2023-12-04 11:08:04,989 - logger - INFO - convert-ctracts_pct-area_2020-to-2010.json already exists in specified download directory (data)
2023-12-04 11:08:04,990 - logger - INFO - tract_conversion_table_2010-2010_raw.csv already exists in specified download directory (data)


In [23]:
## Read in crosswalks: 
def read_json_rows(fp:str) -> list:
    """Read JSON row file to list of dictionaries"""
    output = []
    with open(fp, 'r') as file: 
        for line in file: 
            json_row = json.loads(line.strip())
            output.append(json_row)
    return output

# Convert from 2010 tract values to 2020 tract values (most common use)
map_2010_to_2020 = read_json_rows(os.path.join('data', 'convert-ctracts_pct-area_2010-to-2020.json'))

# Convert from 2020 tract values to 2010 tract values (not sure when we'd ever want to do this, but the file is available)
# map_2020_to_2010 = read_json_rows(os.path.join('data', 'convert-ctracts_pct-area_2020-to-2010.json'))

map_2010_to_2020[:3] 

[{'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020100',
  'GEOID_TRACT_20_overlap': {'01001020100': 99.909, '01001020803': 0.001}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020200',
  'GEOID_TRACT_20_overlap': {'01001020100': 0.049, '01001020200': 99.96}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020300',
  'GEOID_TRACT_20_overlap': {'01001020300': 100.0}}]

#### 2.) Obtain your Census Tract data from multiple years

In this example we will be converting median household income from DE tracts in 2010-2019 to DE tracts in 2020

In [16]:
## Pull your data from census tracts of different years, e.g. median income 

CENSUS_VARS = ['B19013_001E']

## Obtain yearly data from before 2020 redistricting 
YEARS = range(2010,2020)
year_dataframes = []
for y in YEARS: 
    try: 
        mhhi_data = c.acs5.state_county_tract(fields = ['NAME'] + CENSUS_VARS,
                                            state_fips = "10", # Delaware
                                            county_fips = "*",
                                            tract="*",
                                            year=y)
        mhhi_df = pd.DataFrame(mhhi_data)
        mhhi_df['year'] = y
        year_dataframes.append(mhhi_df)
    except Exception as e:
        logger.warning(str(y) + ": " + str(e))

In [139]:
# -- Transform dataframe as needed -- # 

mhhi_df = pd.concat(year_dataframes)
print('Raw data from Census Api:')
display(mhhi_df.head(1))

print('Transformed (Long)')
mhhi_df[['tract_dec', 'county_name', 'state_name']] = mhhi_df['NAME'].str.split(', ', expand=True)
mhhi_df['tract_dec'] = mhhi_df['tract_dec'].str.lstrip('Census Tract ').str.strip()
mhhi_df['county_name'] = mhhi_df['county_name'].str.rstrip('County')
mhhi_df.rename({'state':'state_fips', 'county':'county_fips'}, axis=1, inplace=True)
mhhi_df.drop(['NAME'], axis=1, inplace=True)

# Reorder columns 
mhhi_df = mhhi_df[['state_fips', 'state_name', 'county_fips', 'county_name', 'tract', 'tract_dec', 'year', 'B19013_001E']]

# No NaN's, but -666666666.0 is missing data indicator. Will change to zero.
print(f'Tracts w/ missing income data: {mhhi_df[mhhi_df["B19013_001E"] <= 0].shape[0]}')
mhhi_df['B19013_001E'] = mhhi_df['B19013_001E'].apply(lambda x: 0 if x < 0 else x)

display(mhhi_df)

# Widen 
print('Widen')
mhhi_df_wider = pd.pivot(data=mhhi_df, 
                         index=mhhi_df.filter(regex='state|tract|county').columns,
                        columns=['year'], 
                        values=CENSUS_VARS) 
                        
mhhi_df_wider.columns = [col[1] for col in mhhi_df_wider.columns] # removes the double column index 

# Reset index
mhhi_df_wider.reset_index(inplace=True)
display(mhhi_df_wider)

# Collapse into nested column
print('Collapse (final format)') 
mhhi_df_wider =  mhhi_df_wider.set_index(['state_fips', 'state_name', 'county_fips', 'county_name', 'tract', 'tract_dec'])\
    .apply(lambda row: {str(col):np.round(row_value, 2) for col, row_value in row.items()}, axis=1)\
    .reset_index()\
    .rename({0:'income_raw'}, axis=1) # Before applying 2020 conversions 

mhhi_df_wider

Raw data from Census Api:


,NAME,B19013_001E,state,county,tract,year
0,"Census Tract 169.04, New Castle County, Delaware",70875.0,10,003,016904,2010


Transformed (Long)
Tracts w/ missing income data: 40


,state_fips,state_name,county_fips,county_name,tract,tract_dec,year,B19013_001E
0,10,Delaware,003,New Castle,016904,169.04,2010,70875.0
1,10,Delaware,003,New Castle,980100,9801,2010,0.0
2,10,Delaware,003,New Castle,990100,9901,2010,0.0
3,10,Delaware,005,Sussex,050101,501.01,2010,52569.0
4,10,Delaware,005,Sussex,050103,501.03,2010,72171.0
...,...,...,...,...,...,...,...,...
213,10,Delaware,001,Kent,040202,402.02,2019,74146.0
214,10,Delaware,001,Kent,040203,402.03,2019,66797.0
215,10,Delaware,001,Kent,040900,409,2019,29566.0
216,10,Delaware,001,Kent,041300,413,2019,38288.0


Widen


,state_fips,state_name,county_fips,county_name,tract,tract_dec,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10,Delaware,001,Kent,040100,401,63292.0,70055.0,74211.0,71141.0,61270.0,59094.0,57375.0,63324.0,63826.0,75194.0
1,10,Delaware,001,Kent,040201,402.01,52367.0,54770.0,49288.0,52857.0,51103.0,51423.0,50744.0,50202.0,52065.0,54800.0
2,10,Delaware,001,Kent,040202,402.02,62210.0,67655.0,74167.0,66998.0,68947.0,70877.0,68205.0,66243.0,71384.0,74146.0
3,10,Delaware,001,Kent,040203,402.03,50000.0,54124.0,55648.0,57075.0,68169.0,62209.0,62708.0,64621.0,67159.0,66797.0
4,10,Delaware,001,Kent,040501,405.01,42485.0,47616.0,52351.0,48073.0,51870.0,54833.0,51820.0,49981.0,50244.0,47288.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,10,Delaware,005,Sussex,051702,517.02,56250.0,60956.0,62975.0,61313.0,59236.0,57946.0,65417.0,70291.0,67422.0,62379.0
214,10,Delaware,005,Sussex,051801,518.01,44882.0,45027.0,48954.0,50078.0,50766.0,45842.0,48379.0,49551.0,51576.0,50448.0
215,10,Delaware,005,Sussex,051802,518.02,40518.0,41607.0,33750.0,36758.0,39428.0,38949.0,39029.0,39555.0,41620.0,44835.0
216,10,Delaware,005,Sussex,051900,519,45820.0,45054.0,46384.0,49318.0,47717.0,43659.0,43180.0,47932.0,48761.0,50051.0


Collapse (final format)


,state_fips,state_name,county_fips,county_name,tract,tract_dec,income_raw
0,10,Delaware,001,Kent,040100,401,"{'2010': 63292.0, '2011': 70055.0, '2012': 742..."
1,10,Delaware,001,Kent,040201,402.01,"{'2010': 52367.0, '2011': 54770.0, '2012': 492..."
2,10,Delaware,001,Kent,040202,402.02,"{'2010': 62210.0, '2011': 67655.0, '2012': 741..."
3,10,Delaware,001,Kent,040203,402.03,"{'2010': 50000.0, '2011': 54124.0, '2012': 556..."
4,10,Delaware,001,Kent,040501,405.01,"{'2010': 42485.0, '2011': 47616.0, '2012': 523..."
...,...,...,...,...,...,...,...
213,10,Delaware,005,Sussex,051702,517.02,"{'2010': 56250.0, '2011': 60956.0, '2012': 629..."
214,10,Delaware,005,Sussex,051801,518.01,"{'2010': 44882.0, '2011': 45027.0, '2012': 489..."
215,10,Delaware,005,Sussex,051802,518.02,"{'2010': 40518.0, '2011': 41607.0, '2012': 337..."
216,10,Delaware,005,Sussex,051900,519,"{'2010': 45820.0, '2011': 45054.0, '2012': 463..."


#### 3.) Apply the years crosswalk to your data

In [107]:
# Create map dataframe
df_map_2010_to_2020 = pd.DataFrame(map_2010_to_2020).set_index(['GEOID_TRACT_10'])
display(df_map_2010_to_2020)

,STATENAME,GEOID_TRACT_20_overlap
GEOID_TRACT_10,,
01001020100,Alabama,"{'01001020100': 99.909, '01001020803': 0.001}"
01001020200,Alabama,"{'01001020100': 0.049, '01001020200': 99.96}"
01001020300,Alabama,{'01001020300': 100.0}
01001020400,Alabama,{'01001020400': 99.991}
01001020500,Alabama,"{'01001020501': 100.0, '01001020502': 100.0, '..."
...,...,...
56043000200,Wyoming,"{'56043000200': 99.996, '56043000302': 0.826}"
56043000301,Wyoming,{'56043000301': 100.0}
56043000302,Wyoming,"{'56043000200': 0.004, '56043000302': 99.174}"


In [207]:
# Join with pulled data 
mhhi_df_wider['GEOID_TRACT_10'] = mhhi_df_wider['state_fips'] + mhhi_df_wider['county_fips'] + mhhi_df_wider['tract']

joined = mhhi_df_wider.merge(df_map_2010_to_2020, how='left', left_on=['GEOID_TRACT_10'], right_index=True).drop('STATENAME', axis=1)

print('Joining Crosswalk')
display(joined.head())
### Mul  

## Row-wise apply function (issue w/ creating these row-wise dicts is recombining values from same 2020 tracts scattered between multiple rows)
# def create_conversion_dict(raw_values, overlap_map, dec_round=2):
#     conversion_dict = {}
#     for tract_2020, pct_overlap in overlap_map.items(): 
#         if pct_overlap > 1: 
#             # This would be an older version of the conversion map before I removed \'* 100\' from the pct_overlap function
#             pct_overlap /= 100
#         conversion_dict[tract_2020] = {year:(val * pct_overlap).round(dec_round) for year,val in raw_values.items()}
#     return conversion_dict

# joined['GEOID_TRACT_20_conversion'] = joined.apply(lambda row: create_conversion_dict(row['income_raw'], row['GEOID_TRACT_20_overlap']), axis=1)

## Column-wise function 
def create_conversion_dict(raw_values, overlaps) -> dict: 
    """Create historical data for 2020 Tracts by multiplying past year's values by their respective crosswalk weights"""
    output_dict = {}
    for rv, ov in zip(raw_values, overlaps):
        for tract_2020, pct in ov.items(): 
            # Convert the raw values for the current 2020 census tract
            pct = pct if pct <= 1 else pct / 100
            converted_raw_values = {year:(val*pct).round(2) for year, val in rv.items()}
            # Add values to the output dictionary 
            if tract_2020 in output_dict.keys(): 
                # Add to the values in the current dictionary
                for year in output_dict[tract_2020].keys(): 
                    output_dict[tract_2020][year] += converted_raw_values[year]
            else: 
                output_dict[tract_2020] = converted_raw_values
    return output_dict

print('Create New DF by multiplying year variables by crosswalk weights')
df = pd.DataFrame(create_conversion_dict(joined['income_raw'], joined['GEOID_TRACT_20_overlap'])).T.rename_axis('GEOID_TRACT_20')
# Also collapsing this one
df = df.apply(lambda row: {str(col):np.round(row_value, 2) for col, row_value in row.items()}, axis=1)\
    .reset_index()\
    .rename({0:'B19013_001E'}, axis=1)

df

Joining Crosswalk


,state_fips,state_name,county_fips,county_name,tract,tract_dec,income_raw,GEOID_TRACT_10,GEOID_TRACT_20_overlap
0,10,Delaware,001,Kent,040100,401,"{'2010': 63292.0, '2011': 70055.0, '2012': 742...",10001040100,{'10001040100': 100.0}
1,10,Delaware,001,Kent,040201,402.01,"{'2010': 52367.0, '2011': 54770.0, '2012': 492...",10001040201,"{'10001040201': 99.923, '10001040204': 0.006, ..."
2,10,Delaware,001,Kent,040202,402.02,"{'2010': 62210.0, '2011': 67655.0, '2012': 741...",10001040202,"{'10001040204': 99.994, '10001040205': 99.957,..."
3,10,Delaware,001,Kent,040203,402.03,"{'2010': 50000.0, '2011': 54124.0, '2012': 556...",10001040203,{'10001040203': 99.885}
4,10,Delaware,001,Kent,040501,405.01,"{'2010': 42485.0, '2011': 47616.0, '2012': 523...",10001040501,{'10001040501': 100.0}


Create New DF by multiplying year variables by crosswalk weights


,GEOID_TRACT_20,B19013_001E
0,10001040100,"{'2010': 63292.0, '2011': 70055.0, '2012': 742..."
1,10001040201,"{'2010': 56231.68, '2011': 58961.56, '2012': 5..."
2,10001040204,"{'2010': 62520.47, '2011': 67979.56, '2012': 7..."
3,10003016901,"{'2010': 55466.35, '2011': 60117.56, '2012': 5..."
4,10001040205,"{'2010': 62183.25, '2011': 67625.91, '2012': 7..."
...,...,...
258,10005051702,"{'2010': 56332.32, '2011': 61039.23, '2012': 6..."
259,10005051801,"{'2010': 44882.0, '2011': 45027.0, '2012': 489..."
260,10005051802,"{'2010': 40513.54, '2011': 41602.42, '2012': 3..."
261,10005051900,"{'2010': 45815.42, '2011': 45049.49, '2012': 4..."


#### 4.) Obtain geometries  


In [225]:
import pygris 

tract_geoms = pygris.tracts(year=2020, state='DE').set_index('GEOID')
tract_geoms

county_codes = pygris.counties(year=2020, state='DE') # to get the county names -- not included in the above set
county_codes = county_codes[['COUNTYFP', 'NAME']]

Using FIPS code '10' for input 'DE'
Using FIPS code '10' for input 'DE'


In [232]:
df_geoms = df.merge(tract_geoms[['STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME', 'geometry']], 
                    how='left', left_on=['GEOID_TRACT_20'], right_index=True) \
    .rename({'NAME':'TRACT_DEC'}, axis=1) \
             .merge(county_codes, how='left', on='COUNTYFP') \
    .rename({'NAME':'COUNTY_NAME'}, axis=1) \
    
df_geoms = gpd.GeoDataFrame(df_geoms[['STATEFP', 'COUNTYFP', 'COUNTY_NAME','TRACTCE','B19013_001E','geometry']])

df_geoms

,STATEFP,COUNTYFP,COUNTY_NAME,TRACTCE,B19013_001E,geometry
0,10,001,Kent,040100,"{'2010': 63292.0, '2011': 70055.0, '2012': 742...","POLYGON ((-75.76002 39.29682, -75.75626 39.297..."
1,10,001,Kent,040201,"{'2010': 56231.68, '2011': 58961.56, '2012': 5...","POLYGON ((-75.66581 39.29057, -75.66547 39.290..."
2,10,001,Kent,040204,"{'2010': 62520.47, '2011': 67979.56, '2012': 7...","POLYGON ((-75.63471 39.27722, -75.63469 39.277..."
3,10,003,New Castle,016901,"{'2010': 55466.35, '2011': 60117.56, '2012': 5...","POLYGON ((-75.76010 39.29715, -75.75999 39.297..."
4,10,001,Kent,040205,"{'2010': 62183.25, '2011': 67625.91, '2012': 7...","POLYGON ((-75.61400 39.27423, -75.61211 39.275..."
...,...,...,...,...,...,...
258,10,005,Sussex,051702,"{'2010': 56332.32, '2011': 61039.23, '2012': 6...","POLYGON ((-75.55813 38.45574, -75.55812 38.455..."
259,10,005,Sussex,051801,"{'2010': 44882.0, '2011': 45027.0, '2012': 489...","POLYGON ((-75.70157 38.56074, -75.70041 38.560..."
260,10,005,Sussex,051802,"{'2010': 40513.54, '2011': 41602.42, '2012': 3...","POLYGON ((-75.59122 38.53835, -75.59110 38.538..."
261,10,005,Sussex,051900,"{'2010': 45815.42, '2011': 45049.49, '2012': 4...","POLYGON ((-75.69854 38.52205, -75.69571 38.521..."


#### 5.) Upload to Azure 


In [233]:
## Save locally first to upload 
fp = os.path.join('data', 'DE_2010-2020_med-hh-income_tracts.json')
df_geoms.to_file(fp)

azure_manager.upload_blob(fp)

Blob DE_2010-2020_med-hh-income_tracts.json uploaded successfully.


In [240]:
blob_client = azure_manager.container_client.get_blob_client(os.path.basename(fp))

print(blob_client.url)

https://pipelinemapping.blob.core.windows.net/std-geoms/DE_2010-2020_med-hh-income_tracts.json
